In [ ]:
from typing import List

import kfp
import kfp.dsl as dsl

from kubernetes import client as k8s_client
import uuid

In [ ]:
@dsl.pipeline(
    name="Node group autoscalling",
    description="Pipeline to check if all node groups auto scale to run components"
)
def pipeline(indexes: List[int]):

    with dsl.ParallelFor(indexes) as item:
        vop = dsl.VolumeOp(
            name=uuid.uuid4().hex,
            resource_name=f"my-pvc-{item}",
            modes=dsl.VOLUME_MODE_RWO,
            size="1G"
        )

        cop_cpu_small = dsl.ContainerOp(
            name="cpu-small",
            image="amazon/aws-cli",
            command=["sh", "-c"],
            arguments=["sleep 3m && curl http://169.254.169.254/latest/meta-data/instance-type"],
            pvolumes={"/mnt": vop.volume}
        )
        #cop_cpu_small.add_node_selector_constraint('compute-size', 'cpu-small')
        cop_cpu_small.set_memory_request("10G").set_cpu_request("3")
    
    cop_cpu_small2 = dsl.ContainerOp(
            name="cpu-small",
            image="amazon/aws-cli",
            command=["sh", "-c"],
            arguments=["sleep 3m && curl http://169.254.169.254/latest/meta-data/instance-type"]
        )
    #cop_cpu_small2.add_node_selector_constraint('compute-size', 'cpu-small').after(cop_cpu_small)
    cop_cpu_small2.set_memory_request("10G").set_cpu_request("3")

In [ ]:
if __name__ == '__main__':

    client = kfp.Client()

    # you may have to scale this one till your cluster fails.
    number_of_nodes = 50

    run = client.create_run_from_pipeline_func(
        pipeline_func=pipeline, 
        run_name="node-group-test", 
        experiment_name='scale-test',
        arguments={"indexes":list(range(number_of_nodes))}
    )